In [ ]:
from datetime import date
import random
import time
import yfinance as yf
import pandas as pd

import seaborn as sns

import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm

from numpy.fft import fft, ifft, fftshift
import numpy as np
from numpy import log, sqrt, exp
from numpy import pi, log, exp, real

from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, mean_absolute_percentage_error
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.mixture import GaussianMixture


from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.stats.diagnostic import acorr_ljungbox

import scipy.stats as stats
from scipy.stats import probplot, laplace, norm, t, poisson
from scipy.linalg import solve_banded
from scipy.optimize import minimize, differential_evolution
from scipy.integrate import quad
from scipy.special import roots_laguerre
from scipy.interpolate import interp1d
from scipy.sparse import diags, kron, identity, csr_matrix, eye, csc_matrix
from scipy.sparse.linalg import spsolve
import scipy.sparse as sp
import scipy.sparse.linalg as spla
from scipy.interpolate import RegularGridInterpolator

import statsmodels.api as sm
from statsmodels.nonparametric.kde import KDEUnivariate
from statsmodels.tsa.stattools import adfuller, kpss
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.arima_process import ArmaProcess

#import pymc as pm
#import arviz as az

from tensorflow import keras
#from tensorflow.keras.utils import plot_model

######################################
#from pmdarima import auto_arima
#from diptest import diptest

In [ ]:
class heston_options_value:
    def __init__(self, S0, K, T, r, rho, kappa, sigma, mu, theta, v0, option_type):
        self.S0 = S0
        self.K = K
        self.T = T
        self.r = r
        self.kappa = kappa
        self.theta = theta
        self.sigma = sigma
        self.rho = rho
        self.v0 = v0
        self.mu = mu  # Not used in risk-neutral pricing
        self.option_type = option_type.lower()

    def compute_greeks_closed(self, h=1e-4):
        base_price = self.heston_p1p2()

        # ---- Delta: dPrice/dS0 ----
        S0_up = self.S0 + h
        S0_down = self.S0 - h
        self.S0 = S0_up
        price_up = self.heston_p1p2()
        self.S0 = S0_down
        price_down = self.heston_p1p2()
        delta = (price_up - price_down) / (2 * h)
        gamma = (price_up - 2 * base_price + price_down) / (h**2)
        self.S0 = S0_up - h  # Reset to original

        # ---- Vega: dPrice/dv0 ----
        v0_up = self.v0 + h
        v0_down = self.v0 - h
        self.v0 = v0_up
        price_up = self.heston_p1p2()
        self.v0 = v0_down
        price_down = self.heston_p1p2()
        vega = (price_up - price_down) / (2 * h)
        self.v0 = v0_up - h  # Reset

        # ---- Rho: dPrice/dr ----
        r_up = self.r + h
        r_down = self.r - h
        self.r = r_up
        price_up = self.heston_p1p2()
        self.r = r_down
        price_down = self.heston_p1p2()
        rho = (price_up - price_down) / (2 * h)
        self.r = r_up - h  # Reset

        # ---- Theta: -dPrice/dT ----
        T_up = self.T + h
        T_down = self.T - h
        self.T = T_up
        price_up = self.heston_p1p2()
        self.T = T_down
        price_down = self.heston_p1p2()
        theta = -(price_up - price_down) / (2 * h)
        self.T = T_up - h  # Reset

        return delta, gamma, theta, vega, rho

    def char_func(self, u):
        """
        Characteristic function for log(S_T) under the Heston model.
        """
        i = 1j

        x0 = np.log(self.S0)
        d = np.sqrt((self.rho * self.sigma * u * i - self.kappa)**2 + (self.sigma**2) * (u * i + u**2))
        g = (self.kappa - self.rho * self.sigma * u * i - d) / (self.kappa - self.rho * self.sigma * u * i + d)

        term1 = i * u * x0
        term2 = i * u *self.r * self.T + (self.kappa * self.theta / self.sigma**2) * ((self.kappa - self.rho * self.sigma * u * i - d) * self.T - 2 * np.log((1 - g * np.exp(-d * self.T)) / (1 - g)))
        term3 = (self.v0 / self.sigma**2) * (self.kappa - self.rho * self.sigma * u * i - d) * (1 - np.exp(-d * self.T)) / (1 - g * np.exp(-d * self.T))

        return np.exp(term1 + term2 + term3)

    def compute_P1_P2(self, N=4096, eta=0.25):
        u = np.arange(1, N+1) * eta  # Avoid u=0 to prevent division by zero
        logK = np.log(self.K)

        # Characteristic functions for P1 and P2
        phi_u = self.char_func(u - 1j) / self.char_func(-1j)  # For P1
        phi_v = self.char_func(u)                             # For P2

        integrand_P1 = np.real(np.exp(-1j * u * logK) * phi_u / (1j * u))
        integrand_P2 = np.real(np.exp(-1j * u * logK) * phi_v / (1j * u))

        # Trapezoidal rule (step size eta)
        P1 = 0.5 + (eta / np.pi) * np.sum(integrand_P1)
        P2 = 0.5 + (eta / np.pi) * np.sum(integrand_P2)

        return P1, P2

    def heston_p1p2(self, N=4096, eta=0.25):
        P1, P2 = self.compute_P1_P2(N, eta)

        if self.option_type == 'call':
            price = self.S0 * P1 - self.K * np.exp(-self.r * self.T) * P2
        else:
            price = self.K * np.exp(-self.r * self.T) * (1 - P2) - self.S0 * (1 - P1)

        return price
    ############################################################################################################

    def make_1D_diff_matrices(self, N, d):
        """Builds centered 1st and 2nd derivative matrices with Dirichlet BCs."""
        e = np.ones(N+1)
        D1 = sp.diags([-e, e], [-1, 1], shape=(N+1, N+1)) / (2 * d)
        D2 = sp.diags([e, -2*e, e], [-1, 0, 1], shape=(N+1, N+1)) / (d ** 2)
        D1 = D1.tolil(); D2 = D2.tolil()
        D1[0, :], D1[-1, :] = 0, 0
        D2[0, :], D2[-1, :] = 0, 0
        return D1.tocsr(), D2.tocsr()

    def build_L_heston(self, Nx, Nv, Dx, Dxx, Dv, Dvv, x, v, kappa, theta, sigma, rho, r):
        """
        Builds the full 2D operator L using Kronecker products for the Heston PDE.
        """

        # Term 1: 0.5 * v * d2/dx2
        L1 = sp.kron(Dxx, 0.5 * sp.diags(v))

        # Term 2: 0.5 * sigma^2 * v * d2/dv2
        L2 =sp.kron(sp.eye(Nx+1), 0.5 * sigma**2 * sp.diags(v) @ Dvv)

        # Term 3: (r - 0.5v) * du/dx
        L3 = sp.kron(Dx, sp.diags(r - 0.5 * v) )

        # Term 4: kappa*(theta - v) * du/dv
        L4 = sp.kron(sp.eye(Nx+1), sp.diags(kappa * (theta - v)) @ Dv)

        # Term 5: rho*sigma*v * d2/dx dv
        L5 = sp.kron(Dx, sp.diags(v)) @ sp.kron(sp.eye(Nx+1), rho * sigma * Dv)

        # Term 6: -r * u
        L6 = -r * sp.eye((Nx+1) * (Nv+1))

        L = L1 + L2 + L3 + L4 + L5 + L6
        return L.tocsr()


    def psor_cn_2pde(self, A, b, payoff, omega=1.2, tol=1e-2, max_iter=10):
        """
        Projected SOR method to solve (A u >= b, u >= payoff)
        """
        u = payoff.copy()
        N = len(b)

        for it in range(max_iter):
            u_old = u.copy()

            for i in range(N):
                row_start = A.indptr[i]
                row_end = A.indptr[i + 1]
                Ai = A.indices[row_start:row_end]
                Av = A.data[row_start:row_end]

                a_ii = 0.0
                sum_off_diag = 0.0

                for j, a_ij in zip(Ai, Av):
                    if j == i:
                        a_ii = a_ij  # diagonal
                    else:
                        sum_off_diag += a_ij * u[j]  # use current/old u[j] (Gauss-Seidel style)

                if a_ii == 0:
                    continue  # skip if no diagonal

                # Explicitly include (1 - ω)
                u_new = (1 - omega) * u[i] + (omega / a_ii) * (b[i] - sum_off_diag)
                u[i] = max(payoff[i], u_new)

            error = np.linalg.norm(u - u_old, ord=np.inf)
            if error < tol:
                break
#        else:
#            print("⚠️ PSOR did not converge within max_iter")

        return u

    def pde2_cn(self, exercise_type, exercise_days):
        # Grid parameters
        Nx, Nv, Nt = 80, 40, 100
        S_max, v_max = 3 * self.S0, 3 * self.v0
        x_min, x_max = np.log(1e-4), np.log(S_max)
        v_min = 1e-6

        dt = self.T / Nt

        x = np.linspace(x_min, x_max, Nx + 1)
        v = np.linspace(v_min, v_max, Nv + 1)
        dx = x[1] - x[0]
        dv = (v_max - v_min) / Nv

        X, V = np.meshgrid(x, v, indexing='ij')

        if exercise_type == 'european':
            exercise_binary = np.zeros(Nt + 1)
        elif exercise_type == 'american':
            exercise_binary = np.ones(Nt + 1)
        elif exercise_type == 'bermudan':
            exercise_binary = np.zeros(Nt + 1)
            for i in range(Nt+1):
                if i in exercise_days:
                    exercise_binary[i] = 1

        # Initial condition (European put)
        S = np.exp(X)

        # Terminal payoff
        if self.option_type == 'call':
            U = np.maximum(S - self.K, 0)
        elif self.option_type == 'put':
            U = np.maximum(self.K - S, 0)

        # Flatten U and prepare identity
        U_flat = U.flatten()
        I = sp.eye((Nx + 1) * (Nv + 1))

        Dx, Dxx = self.make_1D_diff_matrices(Nx, dx)
        Dv, Dvv = self.make_1D_diff_matrices(Nv, dv)

        # Build L once
        L = self.build_L_heston(Nx, Nv, Dx, Dxx, Dv, Dvv, x, v, self.kappa, self.theta, self.sigma, self.rho, self.r)

        # Time-stepping loop (Crank-Nicolson)
        A = I - 0.5 * dt * L
        B = I + 0.5 * dt * L

        for n in range(Nt):
            rhs = B @ U_flat

            # Solve A x = rhs
            if exercise_binary[n] == 1:
                payoff = np.maximum(S[:, None] - self.K, 0) if self.option_type == 'call' else np.maximum(self.K - S[:, None], 0)
                payoff_flat = payoff.flatten()
                U_flat = self.psor_cn_2pde(A, rhs, payoff_flat)
            else:
                U_flat = spla.spsolve(A, rhs)

            # Reshape and plot
            U = U_flat.reshape((Nx + 1, Nv + 1))

            # Apply Dirichlet BCs (standard choice: payoff = 0 at boundaries)
            U[0, :] = 0        # x = x_min (S → 0)
            U[-1, :] = U[-1, :] = np.exp(x_max) - self.K if self.option_type == 'call' else 0  # x = x_max (S → ∞)
            U[:, 0] = U[:, 1]  # v = v_min (absorbing or Neumann: du/dv = 0)
            U[:, -1] = U[:, -2]  # v = v_max (same)

            U_flat = U.flatten()

        U = U_flat.reshape((Nx + 1, Nv + 1))

        # Interpolate to get value at (S0, v0)
        interp = RegularGridInterpolator((x, v), U)
        U_price = interp([[np.log(self.S0), self.v0]])[0]

        return U_price